# 演習３：ChatGPTを用いた雑談システム

ChatGPTのAPIを用いて雑談対話を実装します。
ただし、ライセンスキーは各自で取得し、アクセス料金も自己責任とします。

## 必要なライブラリのインストール

はじめに、必要となるライブラリをインストールします。ここではpipを使ってインストールします。

In [1]:
# OpenAIのAPIにアクセスするためのライブラリ
! pip install openai

# 音声認識
! pip install pyaudio
! pip install SpeechRecognition

# 音声合成
! pip install gTTS
! pip install pygame

     -------------------------------------- 320.6/320.6 kB 4.9 MB/s eta 0:00:00
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
     ---------------------------------------- 75.6/75.6 kB ? eta 0:00:00
     -------------------------------------- 409.5/409.5 kB 8.5 MB/s eta 0:00:00
     ---------------------------------------- 77.9/77.9 kB 4.5 MB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
     ---------------------------------------- 1.9/1.9 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Using cached SpeechRecognition-3.10.4-py2.py3-none-any.whl (32.8 MB)
     --------------------------------------- 10.8/10.8 MB 65.1 MB/s eta 0:00:00


## Step 1: OpenAI APIのテスト

OpenAIが提供するAPIをテストします。

In [3]:
# 必要なラブラリを読み込む
import os
import openai

アクセスキーを設定します。アクセスキーは各自で取得してください。ここでは環境変数 "OPEN_API_KEY" にアクセスキーが設定されているとします。

In [4]:
# OpenAIのAPIのアクセスキーを設定
openai.api_key = os.getenv("OPENAI_API_KEY")
ENGINE_NAME = 'gpt-4o'   # GPTのエンジン名

In [ ]:
# 入力を受け付けて応答を生成する
user_input = input()

# OpenAIのAPIを呼び出して、応答を生成する
# 入力は messaages の中に、role と content の２つの要素を持つ辞書型のリストとして与える
# その際、ユーザ発話は role に "user"、システム応答は role に "agent" を与える
completion = openai.ChatCompletion.create(
    engine = ENGINE_NAME,
    messages=[
        {"role": "user", "content": user_input}
    ]
)

# 生成された応答を表示する
system_response = completion.choices[0].message.content
print("システム応答：" + system_response)
# 応答を生成する際のトークン数を表示する
print("トークン数：" + str(completion.usage.total_tokens))

In [ ]:
# 次のユーザ発話を入力する
user_input2 = input()

# 過去のシステム発話を文脈として追加して応答を生成する
# messagesに文脈情報を追加していく
completion = openai.ChatCompletion.create(
    engine = ENGINE_NAME,
    messages=[
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": system_response},
        {"role": "user", "content": user_input2}
    ]
)

# 生成された応答を表示する
system_response2 = completion.choices[0].message.content
print("システム応答：" + system_response2)
# 応答を生成する際のトークン数を表示する
print("トークン数：" + str(completion.usage.total_tokens))

### Step 2: 音声対話システムとしての統合

演習1で実装した方法を用いて、音声対話システムとして動作するように上記の機能を統合します。

In [ ]:
# ライブラリのインポート
import speech_recognition as sr
from gtts import gTTS
import pygame

# 音声認識を関数化
def get_asr():
    
    r = sr.Recognizer()
    r.pause_threshold = 0.5
    
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source) # 背景雑音へ適応する（１秒間）
        print("どうぞ話してください >> ")
        audio = r.listen(source)
    
    try:
        result = r.recognize_google(audio, language="ja-JP")
    except sr.UnknownValueError:
        result = ""
    except sr.RequestError as e:
        result = ""
    
    return result

# 音声合成を関数化
def play_tts(text):
    
    speech = gTTS(text=text, lang="ja")

    try:
        speech.save("./data/test.mp3")
    except Exception as e:
        print('ファイル保存エラー')
    
    pygame.mixer.init()
    pygame.mixer.music.load("./data/test.mp3")
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

    pygame.mixer.music.stop()
    pygame.mixer.quit()

In [ ]:
# 入力発話ならびにシステム応答の文脈を入力するためのリスト
messages = []

# 対話が終了状態に移るまで対話を続ける
while True:
    
    # 音声入力＆音声認識
    result_asr_utterance = get_asr()
    print("ユーザ： " + result_asr_utterance)
    
    # 「終了」がユーザ発話に含まれていれば対話を終了
    if "終了" in result_asr_utterance:
        break
    
    # 応答を生成
    messages.append({"role": "user", "content": result_asr_utterance})
    completion = openai.ChatCompletion.create(
        engine = ENGINE_NAME,
        messages=messages
    )
    
    # 生成された応答を再生する
    system_utterance = completion.choices[0].message.content
    print("システム： " + system_utterance)
    play_tts(system_utterance)

    # システム応答を文脈として追加
    messages.append({"role": "assistant", "content": system_utterance})
    
    print()

# 対話終了
print("対話終了")